<a href="https://colab.research.google.com/github/Tensai-Pro/SRI-PracticeTasks/blob/master/Word%20Embeddings/5)%20BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

from gensim.utils import simple_preprocess

import nltk;
nltk.download('stopwords');
from nltk.corpus import stopwords;

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
df = pd.read_csv('/content/gdrive/MyDrive/Practice Files/train.zip')
df.head(3)

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...


In [4]:
df = df.iloc[:5000]
df.shape

(5000, 3)

# BERT

In [7]:
# !pip install -U tensorflow
# !pip install -U keras
# !pip install -U tensorflow-text

In [8]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [9]:
preprocess_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
encoder_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'

In [10]:
bert_preprocess_model = hub.KerasLayer(preprocess_url)

In [11]:
bert_encoder_model = hub.KerasLayer(encoder_url)

## Тест

In [12]:
samples = df.sample(3)
samples.head()

,Class Index,Title,Description
3389,2,"Canadian Allenby, NCAA champion Moore advance ...","MAMARONECK, NY (CP) - NCAA champion Ryan Moore..."
826,4,Cheap Windows comes under fire,Microsoft's attempt to win over new users with...
2004,4,HHS spends \$228M on IT programs,A preliminary accounting of efforts by the Dep...


In [13]:
text = samples.Description.values[0]
print(text)
print(len(text.split(' ')))

MAMARONECK, NY (CP) - NCAA champion Ryan Moore and Canadian James Allenby were among 32 golfers who advanced with match-play victories Wednesday at the US Amateur. 
27


In [14]:
preprocessed_text = bert_preprocess_model(samples.Description)

In [15]:
print(preprocessed_text.keys())

dict_keys(['input_word_ids', 'input_mask', 'input_type_ids'])


In [16]:
print(preprocessed_text['input_mask'][0])

tf.Tensor(
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], shape=(128,), dtype=int32)


In [17]:
print(preprocessed_text['input_type_ids'][0])

tf.Tensor(
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], shape=(128,), dtype=int32)


In [18]:
print(preprocessed_text['input_word_ids'][0])

tf.Tensor(
[  101  9588 20793  3600  1010  6396  1006 18133  1007  1011  5803  3410
  4575  5405  1998  3010  2508  5297  3762  2020  2426  3590 20601  2015
  2040  3935  2007  2674  1011  2377  9248  9317  2012  1996  2149  5515
  1012   102     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0], shape=(128,), dtype=int32)


In [19]:
bert_results = bert_encoder_model(preprocessed_text)

In [20]:
print(bert_results.keys())

dict_keys(['pooled_output', 'default', 'sequence_output', 'encoder_outputs'])


In [21]:
# 3 предложения по 128 токенов, каждый из которых представлен вектором из 768 эл-тов
print(bert_results['sequence_output'][0])

tf.Tensor(
[[-0.34902045 -0.16801623 -0.00895874 ...  0.10157005  0.6085186
   0.270077  ]
 [ 0.22956926 -0.37728456 -0.37114283 ... -0.05430152  1.0223126
   0.5071656 ]
 [-0.10601289  0.14713055 -0.26956826 ...  0.16980912  0.38116637
   0.6914697 ]
 ...
 [-0.55991286 -0.2207203   0.17960928 ...  0.6556965   0.537485
  -0.21683823]
 [-0.81858176 -0.0624374   0.31444973 ...  0.35581094  0.26957673
  -0.32978716]
 [-0.7829772  -0.1625023   0.3723354  ...  0.446734    0.3604453
  -0.38111085]], shape=(128, 768), dtype=float32)


In [22]:
# представление всего ввода единым вектором
print(bert_results['pooled_output'])

tf.Tensor(
[[-0.5666249  -0.36685196 -0.54855365 ... -0.34336105 -0.6169909
   0.5257471 ]
 [-0.9355428  -0.6286452  -0.98456544 ... -0.92771447 -0.67698216
   0.76043093]
 [-0.849216   -0.46372423 -0.94151634 ... -0.6970219  -0.6992124
   0.56628364]], shape=(3, 768), dtype=float32)


In [23]:
# print(bert_results['default'])

## Построение нейронной модели классификации

In [24]:
# # Bert layers
# text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
# preprocessed_text = bert_preprocess_model(text_input)
# outputs = bert_encoder_model(preprocessed_text)

# # Neural network layers
# l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
# l = tf.keras.layers.Dense(5, activation='softmax', name="output")(l)

# # Use inputs and outputs to construct a final model
# model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [25]:
# model.summary()

In [26]:
# METRICS = [
#       tf.keras.metrics.CategoricalAccuracy(name='sparse_categorical_accuracy'),
#       tf.keras.metrics.Precision(name='precision'),
#       tf.keras.metrics.Recall(name='recall')
# ]

# # model.compile(optimizer='adam',
# #               loss=tf.keras.losses.SparseCategoricalCrossentropy(),
# #               metrics=METRICS)

# model.compile(loss='sparse_categorical_crossentropy', 
#               optimizer='adam',
#               metrics=['sparse_categorical_accuracy'],)

# BERT embeddings 


In [ ]:
batch_size = 600
n = len(df.Description.values)
embeds = np.zeros((n, 768))

for i in range(0, n, batch_size):
  i_end = min(i+batch_size, n)
  print(i_end)
  batch = df.Description.iloc[i:i_end]
  preprocessed = bert_preprocess_model(batch)
  res = bert_encoder_model(preprocessed)
  embeds[i:i_end, :] = res['pooled_output']

600
1200
1800
2400
3000
3600
4200
4800
5000


In [ ]:
embeds.shape

(5000, 768)

In [ ]:
embeds

array([[-0.89392185, -0.55346709, -0.93546826, ..., -0.51121753,
        -0.72716409,  0.87667269],
       [-0.81809491, -0.56045812, -0.94882387, ..., -0.69731843,
        -0.65302438,  0.75921273],
       [-0.58344895, -0.52234983, -0.87617284, ..., -0.44099465,
        -0.61636096,  0.72531563],
       ...,
       [-0.88190961, -0.61667532, -0.94897038, ..., -0.89598143,
        -0.66002029,  0.66592455],
       [-0.83909011, -0.55105168, -0.95318729, ..., -0.59766954,
        -0.657749  ,  0.65339655],
       [-0.6723814 , -0.43944305, -0.98315448, ..., -0.83509392,
        -0.63395101,  0.59584117]])

In [ ]:
np.save('saved_5000_vecs', embeds)

In [ ]:
with open('saved_5000_vecs.npy', 'rb') as f:
  saved_vecs = np.load(f)

saved_vecs == embeds

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

# Классификация

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(embeds, df['Class Index'],
                                                    shuffle=True, random_state=42)

## MultinomialNB

In [ ]:
scaler = MinMaxScaler()

In [ ]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
clf_nb = MultinomialNB().fit(X_train_scaled, y_train)

In [ ]:
y_pred_nb = clf_nb.predict(X_test_scaled)

In [ ]:
acc_score = clf_nb.score(X_test_scaled, y_test) * 100
print(f'Accuracy: {acc_score}%')

Accuracy: 41.839999999999996%


## KNN

In [ ]:
clf_knn = KNeighborsClassifier(n_neighbors=15).fit(X_train_scaled, y_train)

In [ ]:
y_pred_knn = clf_knn.predict(X_test_scaled)

In [ ]:
acc_score = clf_knn.score(X_test_scaled, y_test) * 100
print(f'Accuracy: {acc_score}%')

Accuracy: 68.96%


## SVC

In [ ]:
clf_rbf = SVC(kernel='rbf', gamma=0.5, C=0.1).fit(X_train, y_train)
clf_poly = SVC(kernel='poly', degree=3, C=1).fit(X_train, y_train)

In [ ]:
y_pred_rbf = clf_rbf.predict(X_test)
y_pred_poly = clf_poly.predict(X_test)

In [ ]:
acc_score = clf_rbf.score(X_test, y_test) * 100
print(f'Accuracy: {acc_score}%')

acc_score = clf_poly.score(X_test, y_test) * 100
print(f'Accuracy: {acc_score}%')

Accuracy: 31.6%
Accuracy: 81.76%


## Cosine similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
print(df.Description.iloc[1])
print(df.Description.iloc[2])

Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.


In [ ]:
cosine_similarity([embeds[1]], [embeds[2]])

array([[0.9713779]])

In [ ]:
print(y_train.iloc[10])   # Tech
print(y_train.iloc[4])   # Sport

4
2


In [ ]:
cosine_similarity([X_train[10]], [X_train[2]])

array([[0.88834227]])